# Tutorial - with a Photutils catalog

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import tempfile

Create a temporary directory in which all files will be stored.

In [ ]:
tmpdir = tempfile.TemporaryDirectory(prefix='musex')
tmpdir

## Settings

The settings are specified in a YAML file. MuseX comes with a default settings file (`musex/musex/udf/settings.yaml`) that gives a full example for the UDF, which HST priors.

For this tutorial, we use the settings file from the `tests/` directory. The data inside this directory is extracted from the HDFS v1.24 datasets, and the catalog was created with [Photutils](http://photutils.readthedocs.io/).

In [ ]:
import musex

In [ ]:
DATADIR = os.path.abspath(os.path.join(os.path.dirname(musex.__file__), '..', 'tests', 'data'))
DATADIR

Let's create a settings file in the temp directory, with correct paths.

In [ ]:
settings_file = os.path.join(tmpdir.name, 'settings.yaml')
with open(os.path.join(DATADIR, 'settings.yaml'), 'r') as f:
    out = f.read().format(tmpdir=tmpdir.name, datadir=DATADIR)    
with open(settings_file, 'w') as f:
    f.write(out)
    
print(out)

## Create the MuseX object

This is the main object to manage all the extraction process below. Settings can also be overridden with additional arguments.

In [ ]:
mx = musex.MuseX(settings_file=settings_file, author='SCO')

## Datasets

A MuseX object can contains several *datasets*, where a `DataSet` object gathers all the data from a dataset:

- For MUSE datasets: image, cube, exposure map.
- For other datasets (e.g. HST): images, ...

A MuseX object is tied to a given *Muse dataset* (`mx.muse_dataset`). The other datasets are typically added to the sources object during the extraction.

In [ ]:
# The MUSE dataset
mx.muse_dataset, mx.muse_dataset.cube

In [ ]:
# The other (additional) datasets
mx.datasets

In [ ]:
mx.datasets['test'].images

## Input Catalogs (from source detection, here Photutils)

We have access to a list of input catalogs, defined in the settings file. For the example, the catalog was from the Photutils detection code.

In [ ]:
mx.input_catalogs

In [ ]:
photcat = mx.input_catalogs['photutils']

At this point, the catalog is given as a FITS file defined in the settings, and we need to load it in the database.

In [ ]:
photcat.ingest_input_catalog()

In [ ]:
photcat

### See available columns, table info

This gives some information about our catalog: number of rows, columns, and some metadata.

In [ ]:
photcat.info()

## Intro to `Catalog` objects

In MuseX, `Catalog` objects are wrapping a SQL table, using the [SQLAlchemy](http://docs.sqlalchemy.org/en/latest/index.html) package. SQLAlchemy provides a Pythonic interface to the SQL language, and MuseX also provides some higher-level operations.

For instance, to select in the catalog the sources with an ``ID < 5``:

In [ ]:
res = photcat.select(whereclause=photcat.c.id < 5, 
                     columns=[photcat.idname, photcat.raname, photcat.decname])
res

The result of a selection can be exported as Astropy Table (or the `mpdaf.sdetect.Catalog` wrapper by default).
This shows the complete catalog:

In [ ]:
photcat.select().as_table()

It is possible to do any SQL selection and choose which columns to get, using the SQLAlchemy syntax:

In [ ]:
photcat.select(whereclause=photcat.c.source_sum > 50,
               columns=[photcat.idname, photcat.raname, photcat.decname, 'source_sum', 'area']).as_table()

## User catalogs

Input catalogs are kept immutable, instead users can work on a *"user catalog"*. 

User catalogs must be created with the result of a selection. The mandatory columns are `ID`, `RA` and `DEC` (the other columns from the input catalog can be accessed later with a SQL *join*).

So we can create a user catalog named `my-cat`:

In [ ]:
res = photcat.select(columns=[photcat.idname, photcat.raname, photcat.decname])
mycat = mx.new_catalog_from_resultset('my-cat', res, drop_if_exists=True)

And we can look at the metadata which is stored about our new user catalog:

In [ ]:
mycat.info()

Note that the segmentation map is inherited from the parent input catalog.

* * * 

## Restart and use existing catalog (created at previous step)

All of the above is a setup that has to be done only once. Then all the information is stored in the database. Let's restart with a new MuseX object to check that it works.

In [ ]:
import musex
mx = musex.MuseX(settings_file=settings_file)

User catalogs are stored in `mx.catalogs`:

In [ ]:
mycat = mx.catalogs['my-cat']

And we can verify that our sources are still here:

In [ ]:
mycat.select(limit=2).as_table()

## Preprocessing (segmap, masks, ...)

To do a proper spectrum extraction we need masks of the objects and background. These masks are pre-computed from a segmentation map, and they can be convolved to take into account the PSF of MUSE if the segmap comes from another instrument (e.g. HST).
The masks are also aligned with the MUSE field if needed (rotated and resampled). To do this we **attach a MUSE dataset  to a user catalog**.

In [ ]:
mycat.attach_dataset(mx.muse_dataset, skip_existing=False)

The generated files are stored in the *work directory* and are generated only if needed:

In [ ]:
print('work dir:', mycat.workdir)

os.listdir(mycat.workdir / mx.muse_dataset.name)

In [ ]:
mycat.attach_dataset(mx.muse_dataset, skip_existing=True)

The paths to the masks are stored in the database, and can be modified later if needed (the sky mask is the same for all sources by default, but it is possible to create a sky mask specific to a given source):

In [ ]:
mycat.select(limit=2).as_table()

## MarZ export and import

First, we need to set the `REFSPEC` column to tell which spectrum must be used : (this may change later to allow more flexibility, currently it is possible to set a `refspec` value for each source).

In [ ]:
mycat.update_column('refspec', 'MUSE_PSF_SKYSUB')

In [ ]:
mx.export_marz(mycat)

Once the spectra have been processed in MarZ, the results file can be ingested into MuseX:

In [ ]:
mx.import_marz(os.path.join(DATADIR, 'marz-my-cat-hdfs_full_SCO.mz'), mycat)

In [ ]:
mx.marzcat.info()

In [ ]:
mx.marzcat.select(limit=3).as_table()

## Joining catalogs

When the user catalog is ready to be exported, one need to gather the data from multiple catalogs (the input catalog, MarZ, etc.). This can be done with a SQL *join*.

By default `.join` renames the columns with the format `{catname}_{colname}` to avoid name conflicts. But it is also possible to specify manually the column names (see below).

In [ ]:
res = mycat.join([photcat, mx.marzcat], whereclause=(mx.marzcat.c.catalog == mycat.name))
res.as_table()[:2]

## Select columns for the final catalog

To avoid column name conflicts, for now the user needs to select explicitely the columns and rename if needed. This must be done with SQLAlchemy columns objects:

In [ ]:
photcat.c

In [ ]:
str(photcat.c)

In [ ]:
photcols = [photcat.c[name] for name in ['source_sum', 'source_sum_err', 'area', 'eccentricity', 
                                         'orientation', 'ellipticity', 'elongation', 'version']]

marzcols = [mx.marzcat.c[name] for name in ('FinZ', 'QOP', 'Type', 'Blend', 'Defect', 'Revisit', 'Comment')]

It is possible to rename a column with `.label`:

In [ ]:
photcols[2] = photcols[2].label('pixel_area')

Then we concatenate the column names from the user catalog, MarZ, and the input catalog:

In [ ]:
mycat.c + marzcols + photcols

And we can proceed with the join. Note that by default the join uses the `id` column of each catalog, but it is possible to specify others keys with the `keys` arguments. Also, the `mx.marzcat` table can store the results for multiple catalogs, so we need to select the results for our catalog with the `whereclause`:

In [ ]:
res = mycat.join(
    [mx.marzcat, photcat], 
    whereclause=(mx.marzcat.c.catalog == mycat.name), 
    columns=mycat.c + marzcols + photcols, 
    use_labels=False
).as_table()
res[:3]

## Export sources

In [ ]:
# Set the REFSPEC column to tell which spectrum must be used 
# mycat.update_column('refspec', 'MUSE_PSF_SKYSUB')

Currently, to get a redshift and confidence number in the `Source` objects, MuseX looks at specific columns (`Z` and `Confid`) in the table. So here we rename the columns (we could have done this within the SQL join above, with `.label`):

In [ ]:
res.rename_column('FinZ', 'Z')
res.rename_column('QOP', 'Confid')

And we do the export, here for only one source selected by its ID. We also ask for the creation of a PDF file with plots:

In [ ]:
mx.export_sources(mycat.select_ids([1]), create_pdf=True, size=5, srcvers='0.1', apertures=[0.4])

The output files are placed in the *work directory*:

In [ ]:
outdir = os.path.join(mx.workdir, 'export', mycat.name, mx.muse_dataset.name)
outdir, os.listdir(outdir)

A MPDAF source file has been generated, with all the information both in the header and in the extensions (images, spectra):

In [ ]:
from mpdaf.sdetect import Source
Source.from_file(f'{outdir}/source-00001.fits').info()

A PDF file was generated as well.

In [ ]:
import subprocess
subprocess.run(f'pdftoppm -singlefile {outdir}/source-00001.pdf out && convert out.ppm out.png', shell=True)

from IPython.display import Image
display(Image('out.png'))

In [ ]:
# import webbrowser
# webbrowser.open(f'{outdir}/source-00001.pdf')

In [ ]:
# from IPython.display import IFrame
# display(IFrame(f'{outdir}/source-00001.pdf', width=600, height=300))

In [ ]:
# Cleanup temp directory
tmpdir.cleanup()